In [1]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
import chromadb
#from chromadb.config import Settings
import wget
import pandas as pd
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
import os
import wget
from ast import literal_eval
from rag import RAGCollection

# Test that your OpenAI API key is correctly set as an environment variable
# Note. if you run this notebook locally, you will need to reload your terminal and the notebook for the env variables to be live.

with open('secrets.txt', 'r') as file:
    os.environ["OPENAI_API_KEY"] = file.read().strip()
# Note. alternatively you can set a temporary env variable like this:
import os
import openai


EMBEDDING_MODEL = "text-embedding-3-small"


In [3]:
# # Ignore unclosed SSL socket warnings - optional in case you get these errors
# import warnings
#
# warnings.filterwarnings(action="ignore", message="unclosed", category=ResourceWarning)
# warnings.filterwarnings("ignore", category=DeprecationWarning)

# embeddings_url = 'https://cdn.openai.com/API/examples/data/vector_database_wikipedia_articles_embedded.zip'
# 
# # The file is ~700 MB so this will take some time
# wget.download(embeddings_url)

# import zipfile
# with zipfile.ZipFile("vector_database_wikipedia_articles_embedded.zip","r") as zip_ref:
#     zip_ref.extractall("data")

article_df = pd.read_csv('data/vector_database_wikipedia_articles_embedded.csv')




In [4]:
#Read vectors from strings back into a list
article_df['title_vector'] = article_df.title_vector.apply(literal_eval)
article_df['content_vector'] = article_df.content_vector.apply(literal_eval)

# Set vector_id to be a string
article_df['vector_id'] = article_df['vector_id'].apply(str)

article_df.info(show_counts=True)

chroma_client = chromadb.EphemeralClient() # Equivalent to chromadb.Client(), ephemeral.
# Uncomment for persistent client
# chroma_client = chromadb.PersistentClient()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              25000 non-null  int64 
 1   url             25000 non-null  object
 2   title           25000 non-null  object
 3   text            25000 non-null  object
 4   title_vector    25000 non-null  object
 5   content_vector  25000 non-null  object
 6   vector_id       25000 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.3+ MB


In [5]:

if os.getenv("OPENAI_API_KEY") is not None:
    openai.api_key = os.getenv("OPENAI_API_KEY")
    print ("OPENAI_API_KEY is ready")
else:
    print ("OPENAI_API_KEY environment variable not found")


embedding_function = OpenAIEmbeddingFunction(api_key=os.environ.get('OPENAI_API_KEY'), model_name=EMBEDDING_MODEL)


OPENAI_API_KEY is ready


In [7]:
wikipedia_content_collection = chroma_client.create_collection(name='wikipedia_content', embedding_function=embedding_function)
wikipedia_title_collection = chroma_client.create_collection(name='wikipedia_titles', embedding_function=embedding_function)


# Add the content vectors
wikipedia_content_collection.add(
    ids=article_df.vector_id.tolist(),
    embeddings=article_df.content_vector.tolist(),
)

# Add the title vectors
wikipedia_title_collection.add(
    ids=article_df.vector_id.tolist(),
    embeddings=article_df.title_vector.tolist(),
)



# collection = RAGCollection(embedding_type="openAI", name="RAG")

# collection.add(ids=article_df.vector_id.tolist(), embeddings=article_df.content_vector.tolist())
#
# results = collection.query_collection(query="modern art in Europe", n_results=10)



UniqueConstraintError: Collection wikipedia_content already exists

In [9]:
results = wikipedia_content_collection.query(query_texts="modern art in Europe", n_results=10, include=['distances'])


In [10]:
print(results)

{'ids': [['18310', '4777', '6510', '5082', '3536', '16255', '23717', '21824', '17583', '19867']], 'distances': [[1.92246675491333, 1.9277865886688232, 1.9294931888580322, 1.9332842826843262, 1.9341862201690674, 1.9388872385025024, 1.9419195652008057, 1.9431506395339966, 1.9459826946258545, 1.9493794441223145]], 'metadatas': None, 'embeddings': None, 'documents': None, 'uris': None, 'data': None}


In [11]:
df = pd.DataFrame({
    'id': results['ids'][0],
    'score': results['distances'][0],
    'title': article_df[article_df.vector_id.isin(results['ids'][0])]['title'],
    'content': article_df[article_df.vector_id.isin(results['ids'][0])]['text'],
})

df

,id,score,title,content
3536,18310,1.922467,Lenny Kravitz,"Leonard Albert ""Lenny"" Kravitz (born May 26, 1..."
4777,4777,1.927787,Sublime,Sublime was a popular punk/ska rock band. The ...
5082,6510,1.929493,Gene Pitney,Gene Francis Alan Pitney (17 February 1940 – 5...
6510,5082,1.933284,John Denver,"Henry John Deutschendorf Jr. (December 31, 194..."
16255,3536,1.934186,Georg Danzer,"Georg Franz Danzer (born October 7, 1946 in Vi..."
17583,16255,1.938887,Glen Campbell,"Glen Travis Campbell (April 22, 1936 – August ..."
18310,23717,1.941920,Darin (singer),"Darin Zanyar (born June 2, 1987), known simply..."
19867,21824,1.943151,Taylor Swift,"Taylor Allison Swift (born December 13, 1989) ..."
21824,17583,1.945983,Jon Brion,"Jon Brion (born December 11, 1963) is an Ameri..."
23717,19867,1.949379,Jon Walker,"Jonathan Jacob Walker (born September 17, 1985..."


In [ ]:
c